In [ ]:
import sys
sys.path.append("../")

In [ ]:
import os
#import pandas as pd
import numpy as np
import cv2
#from PIL import Image
import random
#import tensorflow as tf
#import re
#import datetime
#import io
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import pickle
#from sklearn.model_selection import train_test_split
from nltk.metrics.distance import edit_distance
#import string
#from utils import generate_token_index
#from utils import score_prediction, generate_token_index, y_labels, generate_dataset
import json
import keras
import string
import yaml

%matplotlib inline

In [ ]:
with open('../configs/config.yml') as f:
    config = yaml.load(f)

In [ ]:
config['images_folder'] = os.path.join('../', config['images_folder'])
config['labels_file'] = os.path.join('../', config['labels_file'])

In [ ]:
with open(config['labels_file']) as f:
    dataset = json.load(f)

### Select a fraction of the train set

In [ ]:
#select only a fraction
fraction = 0.1
max_len = int(len(dataset['train']) * fraction)

indices = np.random.randint(0, len(dataset['train']), max_len)
dataset['train_subsampled'] = [dataset['train'][j] for j in indices]
#len(dataset)
#dataset = dataset_orig
print(len(dataset['train_subsampled']))

In [ ]:
from data_generators.data_generator import DataGenerator

In [ ]:
train_generator = DataGenerator(config, dataset['train_subsampled'], shuffle=True, use_data_augmentation=False)
val_generator = DataGenerator(config, dataset['val'], shuffle=True, use_data_augmentation=False)

In [ ]:
#[batch_x, batch_y1], batch_y2 = train_generator.__getitem__(0)

In [ ]:
y_size = config['image']['image_size']['y_size']
x_size = config['image']['image_size']['x_size']
max_seq_length = config['network']['max_seq_lenght']
latent_dim = config['network']['latent_dim']
num_decoder_tokens = train_generator.num_decoder_tokens

### Keras model

In [ ]:
#from __future__ import print_function
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation, Bidirectional, concatenate, add, Lambda, Permute
from keras.callbacks import EarlyStopping
#import keras.backend as K
from keras.optimizers import Adam

In [ ]:
#latent_dim = 512  # Latent dimensionality of the encoding space.
#optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
## Define an input sequence and process it.
encoder_inputs = Input(shape=(y_size, x_size, 1), name='input_encoder')

#encoder_inputs = Input(shape=(128, 384, 1), name='input_encoder')

#encoder_inputs = Input(shape=(92, 248, 1), name='input_encoder')

#(7,7)
encoder_layer = Conv2D(16, (7, 7), strides=(1,1), padding='same', use_bias=False, name='conv_1')(encoder_inputs)
encoder_layer = BatchNormalization(name='batch_norm_1')(encoder_layer)
encoder_layer = Activation('relu', name='activation_1')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_1')(encoder_layer)

#(7,7)
encoder_layer = Conv2D(32, (5, 5), padding='same', use_bias=False, name='conv_2')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_2')(encoder_layer)
encoder_layer = Activation('relu', name='activation_2')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_2')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(64, (5, 5), padding='same', use_bias=False, name='conv_3')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_3')(encoder_layer)
encoder_layer = Activation('relu', name='activation_3')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_3')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(128, (3, 3), padding='same', use_bias=False, name='conv_4')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_4')(encoder_layer)
encoder_layer = Activation('relu', name='activation_4')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_4')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(256, (3, 3), padding='same', use_bias=False, name='conv_5')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_5')(encoder_layer)
encoder_layer = Activation('relu', name='activation_5')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_5')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)

conv_shapes = encoder_layer.shape[1:]
timesteps = int(conv_shapes[0]*conv_shapes[1])
num_features = int(conv_shapes[2])

encoder_layer = Reshape((-1, num_features), name='reshape')(encoder_layer)

#encoder
encoder = LSTM(latent_dim, return_state=True, name='lstm_encoder')
_, state_h, state_c = encoder(encoder_layer)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(max_seq_length, num_decoder_tokens), name='input_decoder_teacher_forcing')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_decoder')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

#decoder_dense_time_dist = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'), name='time_distributed_dense')
#decoder_outputs = decoder_dense_time_dist(decoder_outputs)

decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='dense')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
train = True

#batch_size = 128  # Batch size for training.
epochs = 50 # Number of epochs to train for.

# Early stopping  
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

if train == True:    
    # Run training
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    #model.fit_generator(generator = generator epochs=epochs, verbose=1)

    model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=epochs, verbose=1, 
                        callbacks=[])

In [ ]:
if train == True:
    model_json = model.to_json()
    with open("../snapshots/graph.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("../snapshots/weights.h5")

### Inference

In [ ]:
graph_path = '../snapshots/graph.json'
weights_path = '../snapshots/weights.h5'

In [ ]:
from keras.models import model_from_json

#model = load_model("../snapshots/full_model.h5")
with open(graph_path, 'r') as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights(weights_path)

In [ ]:
images_folder = '../datasets/words/'

In [ ]:
test_generator = DataGenerator(config, dataset['test'], shuffle=False, use_data_augmentation=False)

In [ ]:
#change images folder since you are in "notebooks" subfolder
#test_generator.images_folder = '../datasets/words/'

In [ ]:
images_test, labels_test = test_generator.get_full_dataset()

In [ ]:
index = random.randint(0, images_test.shape[0] - 1)
plt.imshow(np.reshape(images_test[index, :, :,:], (test_generator.y_size, test_generator.x_size)), cmap=plt.get_cmap('gray'))
plt.show()
print("label: ", labels_test[index])

In [ ]:
images_test.shape

In [ ]:
# Define inference models 
#encoder
#encoder_inputs_inference = Input(shape=(x_size, y_size, 1), name='input_encoder_inference')
encoder_inference = Model(model.get_input_at(0)[0], model.get_layer("lstm_encoder").output[1:])

#decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_inference = Input(shape=(1, num_decoder_tokens), name='input_decoder_inference')

decoder_lstm = model.get_layer("lstm_decoder")
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

#dense layer
decoder_dense = model.get_layer('dense')
decoder_outputs = decoder_dense(decoder_outputs)

#inference
decoder_inference = Model([decoder_inputs_inference] + decoder_states_inputs, [decoder_outputs] + decoder_states)

### Old decoder

In [ ]:
#reverse_target_char_index = train_generator.reverse_token_indices
#token_indices = train_generator.token_indices

In [ ]:
reverse_target_char_index = test_generator.reverse_token_indices
token_indices = test_generator.token_indices


def decode_sequence(input_seq, max_decoder_seq_length=10):
    # Encode the input as state vectors.
    states_value = encoder_inference.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, token_indices['[']] = 1.

    stop_condition = False
    decoded_sentence = ''
#    while not stop_condition:
    i = 0
    while i < max_decoder_seq_length:
        i += 1
        output_tokens, h, c = decoder_inference.predict( [target_seq] + states_value )
           
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        token_value = np.max(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == ']' or token_value == 0.: 
            #or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
pred_test = []

for ind in range(len(labels_test)):    
#for ind in range(3):    

    #row_pred = []

    input_seq = np.reshape( images_test[ind,:,:], (-1, y_size,  x_size, 1) )
    
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("]", "")
    
    #row_pred.append(words_test[ind])
    #row_pred.append(decoded_sentence)
    
    pred_test.append(decoded_sentence)
    
    #print('Data', words_test[ind], '-', ind, 'out of', len(words_test))
    print('GT:', labels_test[ind])
    print('Decoded:', decoded_sentence, '\n')

### New decoder

In [ ]:
def decode_sequence(input_seq, batch_dim, num_decoder_tokens, max_decoder_seq_length, token_indices, reverse_token_indices):
    # Encode the input as state vectors.
    states_value = encoder_inference.predict(input_seq, batch_size = batch_dim)

    full_seq = np.zeros((batch_dim, 1, num_decoder_tokens))

    target_seq = np.zeros((batch_dim, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[:, token_indices['[']] = 1.
   
    for i in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_inference.predict([np.expand_dims(target_seq, axis=1)] + states_value, 
                                                        batch_size = batch_dim )
    
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[:, -1, :], axis=1)
        #        confidence.append(np.max(output_tokens[:, -1, :]))

        target_seq = np.zeros((batch_dim, num_decoder_tokens))    
        for j in range(batch_dim):
            target_seq[j, sampled_token_index[j]] = 1.

        states_value = [h, c]

        #concatenate with the full sequence array
        full_seq = np.concatenate((full_seq, np.expand_dims(target_seq, axis=1)), axis=1)

    #remove first time element (is empty)
    full_seq = full_seq[:, 1:, :]
    decoded_sentences = []
    
    for i in range(batch_dim):
        sentence = []
        for j in range(full_seq.shape[1]):
            sampled_token_index = np.argmax(full_seq[i, j, :])   
            sentence.append(reverse_token_indices[sampled_token_index])
            #print(sampled_token_index)
    
        decoded_sentences.append(''.join(sentence).replace("]", ""))  
            
    return decoded_sentences

In [ ]:
#test_generator.reverse_token_indices

In [ ]:
def predict(images, batch_size = 512):
    n_images = images.shape[0]
    y_size = images.shape[1]
    x_size = images.shape[2]

    n_batches = (n_images + batch_size - 1) // batch_size
    
    output_list = []
    
    for i in range(n_batches):

#    for i in range(1):

        batch_in, batch_out = (batch_size)* i, (batch_size)* i + batch_size

        if batch_out >= n_images:
            batch_out = n_images

        input_seq = images[batch_in:batch_out, :, :, :]
        batch_dim = batch_out - batch_in
        decoded_sentences = decode_sequence(input_seq, batch_dim, test_generator.num_decoder_tokens, 
                                          test_generator.max_seq_length,
                                          test_generator.token_indices,
                                          test_generator.reverse_token_indices)

        output_list.append(decoded_sentences)
    
    #flatten list
    flattened_list = [item for sublist in output_list for item in sublist]
    
    return flattened_list

In [ ]:
pred_test = predict(images_test)

In [ ]:
def score_prediction(y_true, y_pred):
    words_identified = 0
    characters_identified = 0
    char_tot = 0
#    CER = 0

#    list_accuracy_characters = []
    
    for i in range(len(y_pred)):
        #pred_row = [y_true[i], y_pred[i]] 
        
        #check if date are the same
        #if pred_row[0] == pred_row[1]:
        if y_true[i] == y_pred[i]:

            words_identified += 1
            
#        if len(pred_row[1]) < len(pred_row[0]):
#            pred_row[1] += '-' * (len(pred_row[0]) - len(pred_row[1]))    
#        elif len(pred_row[1]) > len(pred_row[1]):

#            pred_row[1] = pred_row[1][0:len(pred_row[0])]

        #check the number of characters that are the same
 #       print(y_true[i])
 #       print(y_pred[i])
        
        levenshtein_distance = edit_distance(y_true[i], y_pred[i])
        n_char = np.maximum(len(y_true[i]), len(y_pred[i]))
        
        normalized_distance = levenshtein_distance/n_char

        characters_identified += normalized_distance
#        char_tot += n_char
        
#        CER += normalized_distance
        
#        print(len(y_true[i]))
#        for k in range(len(y_true[i])):
#            print()
#            if y_true[i][k] == y_pred[1][k]:
#        characters_identified += 1
#        char_tot += 1

    # array_accuracy_characters = np.asarray(list_accuracy_characters)
    CER = float((characters_identified) / len(y_true))
    WER = (len(y_pred) - words_identified)/len(y_pred) 
        
    return CER, WER
#    return WER

In [ ]:
CER_test, WER_test = score_prediction(labels_test, pred_test)
#WER_test = score_prediction(words_test, pred_test)
print('CER: ', round(CER_test * 100, 3), '%')
print('WER: ', round(WER_test * 100, 3), '%')

# ----------------------- Old code---------------------

### Load filenames and labels

In [ ]:
with open(data_folder + 'gt.json') as f:
    dataset = json.load(f)

In [ ]:
dataset_train_orig = dataset['train']
dataset_val = dataset['val']
dataset_test = dataset['test']

### Select only a subsample for training

In [ ]:
#select only a fraction
fraction = 0.1
max_len = int(len(dataset_train_orig) * fraction)

indices = np.random.randint(0, len(dataset_train_orig), max_len)
dataset_train = [dataset_train_orig[j] for j in indices]
#len(dataset)
#dataset = dataset_orig
print(len(dataset_train))

In [ ]:
train_generator = DataGenerator(dataset_train, 128, 384, 1, batch_size = 64)
val_generator = DataGenerator(dataset_val, 128, 384, 1, batch_size = 64)

[batch_x, batch_y1], batch_y2 = train_generator.__getitem__(0)

In [ ]:
 batch_y2.shape

In [ ]:
y_size = train_generator.y_size
x_size = train_generator.x_size
max_decoder_seq_length = train_generator.max_decoder_seq_length
num_decoder_tokens = train_generator.num_decoder_tokens

In [ ]:
val_generator.max_decoder_seq_length = max_decoder_seq_length

In [ ]:
val_generator.max_decoder_seq_length

In [ ]:
batch_x.shape

In [ ]:
index = random.randint(0, batch_x.shape[0] - 1)
plt.figure(figsize=(9, 9)) 
#plt.imshow(np.reshape(array_images[index, :, :], (y_size, x_size)), cmap=plt.get_cmap('gray'))
plt.imshow(np.reshape(batch_x[index], (128,384)), cmap=plt.get_cmap('gray'))
plt.show()
#print("word = ", words[index])

### Keras model

In [ ]:
#from __future__ import print_function
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation, Bidirectional, concatenate, add, Lambda, Permute
from keras.callbacks import EarlyStopping
#import keras.backend as K
from keras.optimizers import Adam

In [ ]:
latent_dim = 512  # Latent dimensionality of the encoding space.
#optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
## Define an input sequence and process it.
encoder_inputs = Input(shape=(y_size, x_size, 1), name='input_encoder')

#encoder_inputs = Input(shape=(128, 384, 1), name='input_encoder')

#encoder_inputs = Input(shape=(92, 248, 1), name='input_encoder')

#(7,7)
encoder_layer = Conv2D(16, (7, 7), strides=(1,1), padding='same', use_bias=False, name='conv_1')(encoder_inputs)
encoder_layer = BatchNormalization(name='batch_norm_1')(encoder_layer)
encoder_layer = Activation('relu', name='activation_1')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_1')(encoder_layer)

#(7,7)
encoder_layer = Conv2D(32, (5, 5), padding='same', use_bias=False, name='conv_2')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_2')(encoder_layer)
encoder_layer = Activation('relu', name='activation_2')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_2')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(64, (5, 5), padding='same', use_bias=False, name='conv_3')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_3')(encoder_layer)
encoder_layer = Activation('relu', name='activation_3')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_3')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(128, (3, 3), padding='same', use_bias=False, name='conv_4')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_4')(encoder_layer)
encoder_layer = Activation('relu', name='activation_4')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_4')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(256, (3, 3), padding='same', use_bias=False, name='conv_5')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_5')(encoder_layer)
encoder_layer = Activation('relu', name='activation_5')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_5')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)


#(2,2)
#encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
#encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
#encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
#encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)


#(3,3)
#encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
#encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
#encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
#encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)

conv_shapes = encoder_layer.shape[1:]
timesteps = int(conv_shapes[0]*conv_shapes[1])
num_features = int(conv_shapes[2])

encoder_layer = Reshape((-1, num_features), name='reshape')(encoder_layer)

#encoder
encoder = LSTM(latent_dim, return_state=True, name='lstm_encoder')
_, state_h, state_c = encoder(encoder_layer)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(max_decoder_seq_length, num_decoder_tokens), name='input_decoder_teacher_forcing')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_decoder')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense_time_dist = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'), name='time_distributed_dense')
decoder_outputs = decoder_dense_time_dist(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
train = True

#batch_size = 128  # Batch size for training.
epochs = 50 # Number of epochs to train for.

#n_times_aug = 1

# Early stopping  
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

if train == True:    
    # Run training
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

#    model.fit([array_images_train, decoder_input_data_train], decoder_target_data_train, batch_size=batch_size,
#              epochs=epochs, validation_data = ([array_images_val, decoder_input_data_val], decoder_target_data_val),
#              verbose=1, callbacks=[])

    #model.fit_generator(generator = generator epochs=epochs, verbose=1)

    model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=epochs, verbose=1, 
                        callbacks=[])

#    model.fit_generator(generator=train_generator, epochs=epochs, verbose=1, 
#                        callbacks=[])

    
#    model.fit_generator(train_datagen.flow([array_images_train, decoder_input_data_train], decoder_target_data_train, batch_size=batch_size),
#                        steps_per_epoch = n_times_aug * len(array_images_train) / batch_size, epochs=epochs, 
#                        validation_data = val_datagen.flow([array_images_val, decoder_input_data_val], decoder_target_data_val, batch_size=batch_size),
#                        validation_steps =  1, verbose=1, callbacks=[early_stop])

In [ ]:
if train == True:
    model_json = model.to_json()
    with open("../snapshots/graph.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("../snapshots/weights.h5")

In [ ]:
#if train == False:
#model.save('../snapshots/model_IAM.h5')
#model.save_weights("../snapshots/weights.h5")

In [ ]:
if train == False:
    from keras.models import load_model, model_from_json
    json_file = open('../snapshots/graph.json', 'r')
    model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("../snapshots/weights.h5")    
#    model = load_model('../snapshots/model_IAM.h5')

In [ ]:
#model.save_weights("../snapshots/weights.h5")

### Old inference

In [ ]:
images_folder = '../datasets/words/'

In [ ]:
test_generator = DataGenerator(config, dataset['test'], shuffle=False, use_data_augmentation=False)

In [ ]:
#change images folder since you are in "notebooks" subfolder
#test_generator.images_folder = '../datasets/words/'

In [ ]:
images_test, labels_test = test_generator.get_full_dataset()

In [ ]:
index = random.randint(0, images_test.shape[0] - 1)
plt.imshow(np.reshape(images_test[index, :, :,:], (test_generator.y_size, test_generator.x_size)), cmap=plt.get_cmap('gray'))
plt.show()
print("label: ", labels_test[index])

In [ ]:
# Define inference models 
#encoder
#encoder_inputs_inference = Input(shape=(x_size, y_size, 1), name='input_encoder_inference')
encoder_inference = Model(model.get_input_at(0)[0], model.get_layer("lstm_encoder").output[1:])

#decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_inference = Input(shape=(1, num_decoder_tokens), name='input_decoder_inference')

decoder_lstm = model.get_layer("lstm_decoder")
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

#dense layer
decoder_dense = model.get_layer('time_distributed_dense').layer
decoder_outputs = decoder_dense(decoder_outputs)

#inference
decoder_inference = Model([decoder_inputs_inference] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_target_char_index = train_generator.reverse_token_indices
token_indices = train_generator.token_indices


def decode_sequence(input_seq, max_decoder_seq_length=5):
    # Encode the input as state vectors.
    states_value = encoder_inference.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, token_indices['[']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_inference.predict( [target_seq] + states_value )
           
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == ']'): 
            #or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
images_test = images_test[0:1000, ...]
labels_test = labels_test[0:1000]

In [ ]:
images_test.shape

In [ ]:
pred_test = []

for ind in range(len(labels_test)):    
    #row_pred = []

    input_seq = np.reshape( images_test[ind,:,:], (-1, y_size,  x_size, 1) )
    
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("]", "")
    
    #row_pred.append(words_test[ind])
    #row_pred.append(decoded_sentence)
    
    pred_test.append(decoded_sentence)
    
    #print('Data', words_test[ind], '-', ind, 'out of', len(words_test))
    print('GT:', labels_test[ind])
    print('Decoded:', decoded_sentence, '\n')

In [ ]:
def score_prediction(y_true, y_pred):
    words_identified = 0
    characters_identified = 0
    char_tot = 0
#    CER = 0

#    list_accuracy_characters = []
    
    for i in range(len(y_pred)):
        #pred_row = [y_true[i], y_pred[i]] 
        
        #check if date are the same
        #if pred_row[0] == pred_row[1]:
        if y_true[i] == y_pred[i]:

            words_identified += 1
            
#        if len(pred_row[1]) < len(pred_row[0]):
#            pred_row[1] += '-' * (len(pred_row[0]) - len(pred_row[1]))    
#        elif len(pred_row[1]) > len(pred_row[1]):

#            pred_row[1] = pred_row[1][0:len(pred_row[0])]

        #check the number of characters that are the same
 #       print(y_true[i])
 #       print(y_pred[i])
        
        levenshtein_distance = edit_distance(y_true[i], y_pred[i])
        n_char = np.maximum(len(y_true[i]), len(y_pred[i]))
        
        normalized_distance = levenshtein_distance/n_char

        characters_identified += normalized_distance
#        char_tot += n_char
        
#        CER += normalized_distance
        
#        print(len(y_true[i]))
#        for k in range(len(y_true[i])):
#            print()
#            if y_true[i][k] == y_pred[1][k]:
#        characters_identified += 1
#        char_tot += 1

    # array_accuracy_characters = np.asarray(list_accuracy_characters)
    CER = float((characters_identified) / len(y_true))
    WER = (len(y_pred) - words_identified)/len(y_pred) 
        
    return CER, WER
#    return WER

In [ ]:
CER_test, WER_test = score_prediction(labels_test, pred_test)
#WER_test = score_prediction(words_test, pred_test)
print('CER: ', round(CER_test * 100, 3), '%')
print('WER: ', round(WER_test * 100, 3), '%')

In [ ]:
#CER_train, WER_train = score_prediction(words_train, pred_train)
##print('CER: ', round(CER_train * 100, 3), '%')
#print('CER: ', round(CER_train * 100, 3), '%')
#print('WER: ', round(WER_train * 100, 3), '%')